# read in 20CR members and write to net cdfs

In [1]:
import xarray as xr, matplotlib.pyplot as plt
from importlib import reload # need to use this if I edit a function file
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????
import dask.diagnostics # dask allows you to check how long something is taking to load
import climtas # needed to count event statistics with a specified duration

In [21]:
# import custom functions
import sys 
sys.path.append('/home/563/kb6999/Functions') # use this if the function file is in a different directory to the notebook
sys.path.append('/g/data/w48/kb6999/20CR_temp_members/') 

import frequently_used_functions as func
import plotting_functions as fplot
import model_functions as funcM
import reanalysis_functions as funcR
import read_in_members as fmem

In [3]:
# Create a list with all the strings identifying your members
# use a combination of range and zfill to get the leading zeros.
member_list = [str(x).zfill(2) for x in range(1,81)]

In [4]:
member_list = member_list#[slice(0,2)]
member_list

['01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80']

In [26]:
fmem = reload(fmem)

In [ ]:
# read in all the datafiles for each year for one member at a time and save each member (with all years) as a netcdf
for m in member_list:
    path = f'/g/data/ua8/C20C/v3/member_monthly/TMPS/*/*_mem0{m}.nc'
    fmem.read_member(path, m)

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 166 times more chunks
  return self.array[key]


In [16]:
path = f'/g/data/ua8/C20C/v3/member_monthly/TMPS/*/*_mem0{m}.nc'
member_ds_t = xr.open_mfdataset(path, parallel=True)
member_ds_t = member_ds_t.chunk(chunks={'time':-1, 'lat':110, 'lon':110})

In [17]:
member_ds_t

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 256, lon: 512, time: 1992)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T10:30:00 ... 2015-12-16T10:30:00
  * lon        (lon) float32 0.0 0.703 1.406 2.109 ... 357.1 357.8 358.5 359.2
  * lat        (lat) float32 89.46 88.77 88.07 87.37 ... -88.07 -88.77 -89.46
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1992, 2), meta=np.ndarray>
    TMP        (time, lat, lon) float32 dask.array<chunksize=(1992, 110, 110), meta=np.ndarray>
Attributes: (12/13)
    CDI:            Climate Data Interface version 1.9.3 (http://mpimet.mpg.d...
    history:        Sun Dec 23 02:28:05 2018: cdo -v -f nc4 -r -monavg TMPS.1...
    institution:    NOAA ESRL Physical Sciences Division & CU/CIRES \n
    Conventions:    CF-1.6
    platform:       Model
    citation:       <http://onlinelibrary.wiley.com/doi/10.1002/qj.776/abstra...
    ...             ...
    observations:   International Surface Pressure Databank version 4.7
    frequency:      mon
    CDO:            Climate Data Operators version 1.9.3 (http://mpimet.mpg.d...
    comments:       Data is from \n NOAA-CIRES-DOE 20th Century Reanalysis ve...
    main_citation:  <https://doi.org/10.1002/qj.3598>.\n Slivinski et al. 201...
    home_page:      <https://go.usa.gov/XTd>

In [9]:
#rename temperature variable
reanal = xr.Dataset({'tmp': member_ds_t.TMP})
# area weighting 
reanal_w = reanal*np.cos(reanal.lat*(np.pi/180))

In [10]:
# group the data by month (take the monthly mean)
reanal_mon = reanal_w.resample(time='M').mean(dim='time').chunk(chunks={'time':-1, 'lat':110, 'lon':110})

In [11]:
# use functions to calculate the monthly anomalies for the globe
monthly_anom_glob = func.monthly_anomaly(reanal_mon, '1850-01', '1881-01')

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 166 times more chunks
  return self.array[key]


In [18]:
monthly_anom_glob.to_netcdf(f'/g/data/w48/kb6999/20CR_temp_members/R_mon_anom_Glob_tmp{m}.nc')